###### 02/21/2024
### QMSSGR5018
# Assignment: Lab Nº1
### ADV. ANALYTIC TECHNIQUES 
#### Author: Sebastián Urbina Julio

### Table of Contents

* **[0. Importing libraries & data set](#bullet0)**
* **[1. How you think your independent variables will be related to your dependent variable.](#bullet1)**
* **[2. Run a multiple ordinal logistic regression.](#bullet2)**
* **[3. Interpret your results.](#bullet3)**
* **[4. Generate some predicted probabilities.](#bullet4)**
* **[5. What you learned about your hypothesized relationship(s) from this exercise.](#bullet5)**

#### 0. Importing libraries & data set<a class="anchor" id="bullet0"></a>

In [57]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.miscmodels.ordinal_model import OrderedModel

In [58]:
# Setting working directory 
path  = r"/Users/saurbina/Downloads/Bases-CEP-90-1/"
file = "base_90.csv"

df = pd.read_csv(path + file)

df

,id_bu_encuesta,id_bu,encuesta,encuesta_a,encuesta_m,estrato,secu,zona_u_r,region_3,gse,...,elec_pres_144_a,elec_pres_144_b_2,constitucion_19,constitucion_20_a,constitucion_20_b_2,constitucion_34,constitucion_35_a,constitucion_35_b,religion_14,pond
0,793,793-90,90,2023,10,20,90079,1,13,3,...,2.0,NaN,1,2.0,NaN,1,5.0,NaN,1,2.393550
1,796,796-90,90,2023,10,20,90079,1,13,3,...,1.0,NaN,1,2.0,NaN,1,-8.0,NaN,2,2.122179
2,792,792-90,90,2023,10,20,90077,1,13,3,...,NaN,1.0,1,1.0,NaN,1,4.0,NaN,2,2.257310
3,1061,1061-90,90,2023,10,20,90144,1,13,3,...,1.0,NaN,1,1.0,NaN,2,NaN,88.0,3,0.868708
4,1063,1063-90,90,2023,10,20,90144,1,13,4,...,2.0,NaN,1,1.0,NaN,1,-8.0,NaN,2,0.868708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1473,1190,1190-90,90,2023,10,20,90171,1,13,4,...,2.0,NaN,1,2.0,NaN,1,3.0,NaN,2,0.690426
1474,786,786-90,90,2023,10,20,90076,1,13,2,...,NaN,1.0,-8,NaN,-8.0,-8,NaN,88.0,3,1.946939
1475,510,510-90,90,2023,10,12,90011,1,8,3,...,2.0,NaN,1,2.0,NaN,1,-8.0,NaN,2,0.765861
1476,512,512-90,90,2023,10,12,90011,1,8,3,...,3.0,NaN,1,2.0,NaN,2,NaN,88.0,2,0.382931


#### 1. How you think your independent variables will be related to your dependent variable.<a class="anchor" id="bullet1"></a>

For the following analysis, I will use a Chilean survey dataset created by the CEP Institute. The universe of the study was all the individuals
above 17 years old and residents of the country, achieving a sample of 1467 individuals. They interviewed each of them in their homes, and the
sample method was stratified, random, and probabilistic in each of its three stages. I chose this dataset because it is among the most
prestigious public opinion surveys, having high methodological standards.

In this sense, we want to see how the perception of the country (Chile) is progressing; stagnant or declining is affected by sex, age, and the presidential vote. Furthermore, in that sense, I hypothesize that sex and age don't affect the perception of the country, but the vote preference has an impact on the dependent variable; this is because if you voted for the actual president, you would have a better perception in the country today but in the other hand if you vote for the other candidate your perception would be worst. 

To test this hypothesis, I will use four variables. The dependent variable name, **"perception"**, is an ordinal variable with three options: progressing, stagnant, and declining. For the independent variables, I will use sex and age as control variables, and the presidential preference name **"vote_preference"** is a recorded variable that has four options, the two candidates, null and blank. For this example, I recoded the variable into a dummy variable, leaving only the candite option because these are the characteristics we are testing. 

Seeing the outcome of the dependent variable, we will use an ordinal logit. This is because when the dependent variable has more than two categories, and the values of each category have a meaningful sequential order where a value is indeed 'higher' than the previous one, then you can use ordinal logit. 

In [59]:
my_tab = pd.crosstab(index=df["percepcion_4"],
                              columns="count")

print(my_tab)

col_0         count
percepcion_4       
-9                4
-8               14
 1              158
 2              767
 3              535


In [60]:
conditions = [
    (df["percepcion_4"] == 1) ,
    (df["percepcion_4"] == 2),
    (df["percepcion_4"] == 3)
]

choices = [3, 2, 1]
#3 = progressing, 2 = stagnant, and 1 = declining

df['perception'] = np.select(conditions, choices, default=np.nan)

my_tab = pd.crosstab(index=df["perception"],
                              columns="count")

df = df.dropna(subset=['perception'])

print(my_tab)

col_0       count
perception       
1.0           535
2.0           767
3.0           158


In [61]:
df['sexo'].value_counts() 

2    923
1    537
Name: sexo, dtype: int64

In [62]:
conditions = [
    (df["sexo"] == 1) ,
    (df["sexo"] == 2)
]

choices = [0, 1]
#1 = female, 0 = male

df['sex'] = np.select(conditions, choices, default=np.nan)

my_tab = pd.crosstab(index=df["sex"],
                              columns="count")

print(my_tab)

col_0  count
sex         
0.0      537
1.0      923


/var/folders/zh/x7zbl54957s8qx431xxympk40000gn/T/ipykernel_68307/2545955414.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sex'] = np.select(conditions, choices, default=np.nan)


In [63]:
df['edad'].value_counts() 

25    37
65    37
48    35
60    35
50    34
      ..
93     2
91     1
90     1
92     1
94     1
Name: edad, Length: 76, dtype: int64

In [64]:
df['age'] = df['edad']
my_tab1 = pd.crosstab(index=df["age"],
                              columns="count")

print(my_tab1)

col_0  count
age         
18        15
19        14
20        21
21        18
22        25
..       ...
90         1
91         1
92         1
93         2
94         1

[76 rows x 1 columns]


/var/folders/zh/x7zbl54957s8qx431xxympk40000gn/T/ipykernel_68307/3090339600.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['age'] = df['edad']


In [65]:
df['elec_pres_144_a'].value_counts() 

 1.0    543
 2.0    338
-9.0    134
 4.0    119
 3.0     25
-8.0     25
Name: elec_pres_144_a, dtype: int64

In [66]:
conditions = [
    (df["elec_pres_144_a"] == 1) ,
    (df["elec_pres_144_a"] == 2)
]

choices = [1, 0]
#1 = Gabriel Boric,  0 = J. Kast

df['vote_preference'] = np.select(conditions, choices, default=np.nan)

my_tab = pd.crosstab(index=df["vote_preference"],
                              columns="count")

print(my_tab)

df = df.dropna(subset=['vote_preference'])


col_0            count
vote_preference       
0.0                338
1.0                543


/var/folders/zh/x7zbl54957s8qx431xxympk40000gn/T/ipykernel_68307/4102683619.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['vote_preference'] = np.select(conditions, choices, default=np.nan)


#### 2. Run a multiple ordinal logistic regression.<a class="anchor" id="bullet2"></a>

In [67]:
mod_prob = OrderedModel(df['perception'],
                        df[['sex', 'age', 'vote_preference']],
                        df='logit')

res_prob = mod_prob.fit(method='bfgs')
res_prob.summary()

Optimization terminated successfully.
         Current function value: 0.891447
         Iterations: 16
         Function evaluations: 22
         Gradient evaluations: 22


<class 'statsmodels.iolib.summary.Summary'>
"""
                             OrderedModel Results                             
==============================================================================
Dep. Variable:             perception   Log-Likelihood:                -785.36
Model:                   OrderedModel   AIC:                             1581.
Method:            Maximum Likelihood   BIC:                             1605.
Date:                Fri, 23 Feb 2024                                         
Time:                        11:10:42                                         
No. Observations:                 881                                         
Df Residuals:                     876                                         
Df Model:                           3                                         
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
sex                -0.0548      0.080     -0.689      0.491      -0.211       0.101
age                -0.0047      0.002     -2.127      0.033      -0.009      -0.000
vote_preference     1.0260      0.085     12.040      0.000       0.859       1.193
1.0/2.0            -0.0871      0.136     -0.642      0.521      -0.353       0.179
2.0/3.0             0.5201      0.039     13.193      0.000       0.443       0.597
===================================================================================
"""

#### 3. Interpret your results.<a class="anchor" id="bullet3"></a>

In [103]:
exp_coef = np.exp(res_prob.params)
print(exp_coef)

sex                0.946687
age                0.995288
vote_preference    2.789929
1.0/2.0            0.916597
2.0/3.0            1.682258
dtype: float64


Compared to males, females, on average and holding all other variables constant, have 5.48% lower odds of being in a higher category of perception of the country versus a lower category. However, this coefficient is not statistically significant.

On the other hand, for each additional year of age, on average and holding all other variables constant, the odds of being in a higher category of perception of the country versus a lower category decrease by 0.47%. This coefficient is statistically significant.

Finally, being a voter of the actual president, compared to someone who voted for the opposing candidate, on average and holding all other variables constant, increases the odds of being in a higher category of perception of the country versus a lower category by 2.6%. This coefficient is statistically significant.

#### 4. Generate some predicted probabilities.<a class="anchor" id="bullet4"></a>

In [100]:
predicted = res_prob.model.predict(res_prob.params, exog=df[['age', 'sex', 'vote_preference']])
predicted = pd.DataFrame(predicted, columns=['perception_dum_1', 'perception_dum_2', 'perception_dum_3'])
predicted


,perception_dum_1,perception_dum_2,perception_dum_3
0,0.988358,0.011603,3.891330e-05
1,0.790786,0.202852,6.361871e-03
2,0.986746,0.013207,4.791150e-05
3,0.971602,0.028230,1.671611e-04
4,0.722406,0.266058,1.153532e-02
...,...,...,...
876,0.603064,0.370961,2.597529e-02
877,0.958701,0.040984,3.153466e-04
878,0.941960,0.057470,5.695633e-04
879,0.982532,0.017393,7.491263e-05


In [101]:
preds_vote = pd.DataFrame({'vote_preference': df['vote_preference'], 'X1': predicted['perception_dum_1'], 'X2': predicted['perception_dum_2'], 'X3': predicted['perception_dum_3']})
preds_vote_means = preds_vote.groupby(['vote_preference']).mean()
preds_vote_means.head()


,X1,X2,X3
vote_preference,,,
0.0,0.919270,0.076878,0.003852
1.0,0.912854,0.083134,0.004011


#### 5.  What you learned about your hypothesized relationship(s) from this exercise.<a class="anchor" id="bullet5"></a>

The hypotheses we had were that sex and age don't affect the perception of the country, but the vote preference has an impact on the dependent variable. This is because if you voted for the current president, you would have a better perception of the country today, but on the other hand, if you voted for the other candidate, your perception would be worse.

In that sense, two out of the three hypotheses were correct: sex does not affect the perception of the country, and the candidate for whom you voted has an impact on the perception of the country. However, we had an interesting outcome when testing age; we found that age does have an impact on the perception of the country. The coefficients are not too big, making me think that these results make sense with reality, particularly because personal perception of the country is a variable that is affected by many other factors.